In [1]:
import sympy
from openLoop.utilities.ipynb.displays import *
from openLoop.utilities.ipynb.ipy_sympy import *
import scipy.linalg


import numpy.testing as np_test
import declarative

from test_SVD import SVD_gen_check, gen_rand_unitary
from openLoop.system import DAG_algorithm
from openLoop.system import SRE_matrix_algorithms
from openLoop.system import scisparse_algorithm

asavefig.org_subfolder = 'plots'

Populating the interactive namespace from numpy and matplotlib
Sympy version:  1.0


In [2]:
from functools import reduce
def SVD_compare_error(
    N = 10,
    length = 10,
    solvers = [
        scisparse_algorithm, 
        DAG_algorithm
    ],
):
    U = gen_rand_unitary(N = N, length = length)
    V = gen_rand_unitary(N = N, length = length)

    seq = dict()
    req = dict()
    edge_map = dict()
    S_diags = []
    for idx in range(N):
        s_diag = 10**(-5 + 10 * np.random.random(length))
        edge_map[idx, idx] = s_diag
        S_diags.append(s_diag)
        seq[idx] = set([idx])
        req[idx] = set([idx])
    S = seq, req, edge_map
    condition = reduce(np.maximum, S_diags) / reduce(np.minimum, S_diags)

    M = SRE_matrix_algorithms.matrix_mult_sre(
        SRE_matrix_algorithms.matrix_mult_sre(U, S), V
    )

    SRE_matrix_algorithms.check_sre(M)

    print("SPARSITY FRAC: ", SRE_matrix_algorithms.SRE_count_sparsity(M))

    inputs_set = set(range(N))
    outputs_set = set(range(N))

    def solve(solver):
        Mseq, Mreq, Medge_map = SRE_matrix_algorithms.copy_sre(M)
        print(solver)
        sbunch = solver.inverse_solve_inplace(
            seq = Mseq,
            req = Mreq,
            edge_map = Medge_map,
            inputs_set = inputs_set,
            outputs_set = outputs_set,
            verbose = True,
            negative = False,
        )

        Minv = sbunch.seq, sbunch.req, sbunch.edge_map
        SRE_matrix_algorithms.check_sre(M)
        SRE_matrix_algorithms.check_sre(Minv)

        Meye = SRE_matrix_algorithms.matrix_mult_sre(M, Minv)
        #pprint(Meye)
        em1_ll = dict()
        em0_ll = dict()
        for (k_t, k_f), edge in Meye[2].items():
            if (k_t == k_f):
                em1_ll[k_t, k_f] = -np.log10(np.maximum(abs(edge - 1), 1e-30))
            else:
                em0_ll[k_t, k_f] = -np.log10(np.maximum(abs(edge), 1e-30))
        
        all_e0 = []
        all_e1 = []
        
        for k, e in em0_ll.items():
            all_e0.append(e)
        for k, e in em1_ll.items():
            all_e1.append(e)
            
        if all_e0:
            all_e0 = np.concatenate(all_e0)
        else:
            all_e0 = []
            
        if all_e1:
            all_e1 = np.concatenate(all_e1)
        else:
            all_e1 = []
        return declarative.Bunch(
            solver = solver,
            Meye = Meye,
            em1_ll = em1_ll,
            em0_ll = em0_ll,
            all0 = all_e0,
            all1 = all_e1,
        )
    sdict = dict()
    for solver in solvers:
        sdict[solver] = solve(solver)
    sdict['condition'] = condition
    return sdict
    #pprint(em1_ll)
    #pprint(em0_ll)


In [3]:
r = SVD_compare_error(
    N = 100,
    length = 100,
)

SPARSITY FRAC:  0.0455
<module 'openLoop.system.scisparse_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/scisparse_algorithm.py'>
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 100
TRIVIAL STAGE, REMAINING 81
BADGUY STAGE, REMAINING 81


In [4]:
b1 = r[DAG_algorithm]
b2 = r[scisparse_algorithm]

In [5]:
Meye1 = b1.Meye[2]
Meye2 = b2.Meye[2]

diag = []
offdiag = []
diag1 = []
offdiag1 = []
diag2 = []
offdiag2 = []
offdiag2x = []
for (k1, k2), e1 in Meye1.items():
    e2 = Meye2[k1, k2]
    e_diff = e1 - e2
    if k1 == k2:
        diag.append(e_diff)
        diag1.append(e1)
        diag2.append(e2)
    else:
        offdiag.append(e_diff)
        offdiag1.append(e1)
        offdiag2.append(e2)
        offdiag2x.append(b2.em0_ll[k1, k2])
diag = np.concatenate(diag)
offdiag = np.concatenate(offdiag)
diag1 = np.concatenate(diag1)
offdiag1 = np.concatenate(offdiag1)
diag2 = np.concatenate(diag2)
offdiag2 = np.concatenate(offdiag2)
offdiag2x = np.concatenate(offdiag2x)

In [6]:
axB = mplfigB(Nrows=4)
Nbins = 30

_ = axB.ax0.hist(b1.all0, normed = 1, bins = Nbins)
_ = axB.ax0.hist(b1.all1, normed = 1, alpha = .5, bins = Nbins)
axB.ax0.set_yscale('log')
print(np.min(b1.all0), np.max(b1.all0))
print(np.min(b1.all1), np.max(b1.all1))
axB.ax0.set_xlim(0, 30)

_ = axB.ax1.hist(b2.all0, normed = 1, bins = Nbins)
_ = axB.ax1.hist(b2.all1, normed = 1, alpha = .5, bins = Nbins)
print(np.min(b2.all0), np.max(b2.all0))
print(np.min(b2.all1), np.max(b2.all1))
axB.ax1.set_yscale('log')
axB.ax1.set_xlim(0, 30)

_ = axB.ax2.hist(-np.log10(np.maximum(abs(offdiag), 1e-30)), normed = 1, bins = Nbins)
_ = axB.ax2.hist(-np.log10(np.maximum(abs(diag), 1e-30)), normed = 1, alpha = .5, bins = Nbins)
#axB.ax2.set_yscale('log')
axB.ax2.set_xlim(0, 30)

_ = axB.ax3.hist(np.log10(r['condition']))

axB.save('comparison')

2.00473878661 30.0
6.39736623699 30.0
0.574637681027 30.0
5.4444982617 30.0
figure: plots/comparison.png
[[file:plots/comparison.png]]


In [7]:
r['condition']

array([  5.64929553e+09,   6.86917623e+09,   6.66087936e+09,
         7.36407485e+09,   6.37831491e+09,   7.56148951e+09,
         6.36606137e+09,   5.57843302e+09,   7.71483921e+09,
         4.54774468e+09,   7.28426121e+09,   7.28079239e+09,
         7.26596526e+09,   6.77867418e+09,   6.45747157e+09,
         8.07738364e+09,   5.73688225e+09,   3.34704216e+09,
         5.97136001e+09,   7.99097621e+09,   6.98511614e+09,
         5.09528594e+09,   5.83754110e+09,   5.42145245e+09,
         5.66990594e+09,   8.39435207e+09,   2.65594696e+09,
         4.11619202e+09,   8.08450927e+09,   3.27898562e+09,
         4.31415665e+09,   8.63333057e+09,   4.77582598e+09,
         8.25003815e+09,   7.02764733e+09,   3.26736579e+09,
         5.14682841e+09,   6.34235561e+09,   2.58849636e+09,
         6.33777635e+09,   5.45342018e+09,   9.35470632e+09,
         8.16507905e+09,   3.29011795e+09,   9.57689709e+09,
         9.25552516e+09,   6.91679506e+09,   4.64201891e+09,
         6.35200671e+09,

In [8]:
idx_max = np.argmax(np.log10(abs(offdiag2)))
print(-np.log10(abs(offdiag2[idx_max])))
print(offdiag2x[idx_max])


0.574637681027
0.574637681027


/home/mcculler/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log10
  if __name__ == '__main__':


In [9]:
idx_max = np.argmax(np.log10(abs(offdiag1)))
print(-np.log10(abs(offdiag1[idx_max])))

2.00473878661


/home/mcculler/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in log10
  if __name__ == '__main__':
